In [3]:
import pandas as pd
from pymongo import MongoClient
import configparser
import gzip
import json
import numpy as np
import matplotlib as plt
import tqdm

Для начала загрузим локально сохраненный датасет git

In [4]:
columns = ['_id',
 'id',
 'name',
 'username',
 'state',
 'avatar_url',
 'web_url',
 'created_at',
 'bio',
 'location',
 'public_email',
 'skype',
 'linkedin',
 'twitter',
 'website_url',
 'organization',
 'last_sign_in_at',
 'confirmed_at',
 'last_activity_on',
 'email',
 'theme_id',
 'color_scheme_id',
 'projects_limit',
 'current_sign_in_at',
 'identities',
 'can_create_group',
 'can_create_project',
 'two_factor_enabled',
 'external',
 'private_profile',
 'is_admin',
 'emails',
 'projects']

In [6]:
n = 7
df = pd.DataFrame(columns=columns)
for i in range(0,n):
  print(i)
  with open(f'/content/drive/MyDrive/bd_git/git_{i}.json', 'rb') as f:
    df_el = pd.read_json(f, compression='zip')
  df = pd.concat([df, df_el])

0
1
2
3
4
5
6


In [ ]:
df

Посмотрим на признак projects для первого человека в таблице

In [8]:
pd.json_normalize(pd.json_normalize(df['projects']).iloc[0]).head(6)

,id,description,name,name_with_namespace,path,path_with_namespace,created_at,default_branch,tag_list,ssh_url_to_repo,...,languages.C#,languages.Smalltalk,languages.Python,owner,permissions.group_access.access_level,permissions.group_access.notification_level,languages.Java,languages.PHP,languages.Vue,languages.HTML
0,198.0,,HSE Companion,Егор Аникеев / HSE Companion,hse-companion,edanikeev/hse-companion,1.583874e+12,master,[],git@git.miem.hse.ru:edanikeev/hse-companion.git,...,92.32,7.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1048.0,,Zulip bot,Егор Аникеев / Zulip bot,zulip-bot,edanikeev/zulip-bot,1.591369e+12,master,[],git@git.miem.hse.ru:edanikeev/zulip-bot.git,...,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3414.0,,onvif-3-2,2020-21 Мультимедиа-технологии / Егор Аникеев ...,onvif-3-2,2020-21-multimedia-technologies/edanikeev/onvi...,1.617007e+12,master,[],git@git.miem.hse.ru:2020-21-multimedia-technol...,...,NaN,NaN,100.0,NaN,50.0,3.0,NaN,NaN,NaN,NaN
3,429.0,,HSE Companion Android Internet Library,19290 Разработка мобильного приложения для инт...,hse-companion-android-internet-library,19290/hse-companion-android-internet-library,1.586025e+12,master,[],git@git.miem.hse.ru:19290/hse-companion-androi...,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN
4,363.0,,Hse Companion The App,19290 Разработка мобильного приложения для инт...,hse-companion-the-app,19290/hse-companion-the-app,1.585858e+12,master,[],git@git.miem.hse.ru:19290/hse-companion-the-ap...,...,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,359.0,,HSE Companion Server,19290 Разработка мобильного приложения для инт...,hse-companion-server,19290/hse-companion-server,1.585856e+12,master,[],git@git.miem.hse.ru:19290/hse-companion-server...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.68,18.75,4.57


Заметим, что база данных помимо учебных проектов, которые нам интересны, содержит так же и личные. Нам нужно отфильтровать их, взяв только учебные

In [9]:
pd.json_normalize(pd.json_normalize(df['projects']).iloc[0])[['namespace.kind', 'namespace.name']].head(6)

,namespace.kind,namespace.name
0,user,Егор Аникеев
1,user,Егор Аникеев
2,group,Егор Аникеев
3,group,19290 Разработка мобильного приложения для инт...
4,group,19290 Разработка мобильного приложения для инт...
5,group,19290 Разработка мобильного приложения для инт...


Первая гипотеза: \
Все или почти все проекты являются учебными, если namespace.kind == group

In [10]:
n = 7
df_namespace = pd.DataFrame(columns=['namespace.kind', 'namespace.name'])
for i in range(n):
    for j in tqdm.tqdm(range(df[442 * i: 442 * i + 442].shape[0])):
        try:
            df_project = pd.json_normalize(pd.json_normalize(df[442 * i: 442 * i + 442]['projects']).iloc[j])[['namespace.kind', 'namespace.name']]
            df_project = df_project[~df_project['namespace.kind'].isnull()]
            df_namespace = pd.concat([df_namespace, df_project])
        except:
            pass
    print(f'{i+1}/{7}')

100%|██████████| 442/442 [06:46<00:00,  1.09it/s]


1/7


100%|██████████| 442/442 [02:44<00:00,  2.69it/s]


2/7


100%|██████████| 442/442 [12:20<00:00,  1.68s/it]


...

5/7


100%|██████████| 442/442 [01:48<00:00,  4.09it/s]


6/7


100%|██████████| 442/442 [00:36<00:00, 11.96it/s]

7/7


In [12]:
df_namespace.value_counts().head(20)

namespace.kind  namespace.name                      
group           БИВ20-Микропроект                       26560
                ivt21-miniproject                        2411
                python-1 (Проектный семинар 2022)        1385
                python-3 (Проектный семинар 2022)         863
                python-2 (Проектный семинар 2022)         856
                19102 Телецентр МИЭМ                      786
                NVR                                       657
                Практическое задание Stepik 2.6 (КГ)      415
                400 Цифровой след аналитика               336
                Проектный семинар 2021                    321
                Zulip                                     309
                Odoo                                      306
                19101 Решебники                           283
                212                                       280
                318 VR тренажер для яхтсменов             273
                2

Судя по данным выше uипотеза не подтвердилась

Заметим, что названия всех учебных проектов начинаются с их номеров и заканчиваются названием, поэтому мы можем попробовать отфильтровать по первому и последнему символам

In [33]:
for name in set(df_namespace[df_namespace['namespace.name'].apply(
                                      lambda x: x[0].isdigit() 
                                      and x[-1].isalpha())]['namespace.name']):
    print(name)

19048 Разработка программного обеспечения, реализующего безопасную почтовую систему в сети Интернет
547 Робот гид
313 Разработка программного обеспечения для изучения лабораторных работ по механике в виртуальной реальности
19104 Облачный видеорекордер
240 Поисковик компетенций
564 Разработка методических указаний по выполнению курсовой работы по Алгоритмизации и программированию для студентов 1 курса ОП ИВТ
542 Методы анализа больших данных
401 Платформа для интеграции экспертной самостоятельной и пиринговой оценки эссе
298 Отслеживание положения лица и глаз относительно камеры для системы прокторинга
857 Разработка интеллектуального робота помощника
657 Разработка высокопроизводительных методов глобальной оптимизации
538 Интерактивная LED панель с динамическим управлением
568 Разработка системы учета переходов студентов на дистанционное обучение и выхода с дистанционного обучения. Информирование преподавателей и студентов
511 Разработка программной части системы управления микроклимат

Проекты отфильтровались верно

Также для составления проектного рейтинга нам понадобится полная информация о коммитах в репрозиториях учебных проектов. Данную информацию можно найти в следующих полях:

In [42]:
pd.json_normalize(pd.json_normalize(df['projects']).iloc[0])['commits'] #для примера взяты коммиты по проектам первого человека в бд

0       [{'id': '9fd074b9ee403928e5ad25bc688b6a898167e...
1       [{'id': 'cc88927f54aea243825ce22fbc39c86c84447...
2       [{'id': '897a6477f86d222d19f267cd6080a2ecff5c6...
3                                                      []
4       [{'id': '3a117b99a49671066f9ed92c97ffb037e6e05...
                              ...                        
3140                                                  NaN
3141                                                  NaN
3142                                                  NaN
3143                                                  NaN
3144                                                  NaN
Name: commits, Length: 3145, dtype: object